In [1]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"

RANDOM_SEED = 42 # for reproducibility
MAX_K_CLUSTERS = 700

# Preparazione

In [2]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=0077a0730c914adaed6d487354ca260abb4bc2464dc1ecbbc0775d35a5c9aec2
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [3]:
!ngrok authtoken 2Tn9uykXlYRjLcSLIgmv1iLdEvG_3gqA9M3V4hLGqH9N3JWEr

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
!apt install openjdk-8-jdk-headless -qq

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 120828 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u382-ga-1~22.04.1_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u382-ga-1~22.04.1) ...

In [5]:
from pyngrok import ngrok
import os
import re

In [6]:
os.environ["JAVA_HOME"] = JAVA_HOME

In [7]:
# Connect this colab to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
# Install PySpark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=74ceb854447f028de753918b52051419eb1ad1ea1cd80f785f8b3268ee03dae4
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [9]:
# Import libraries
# POI CAPISCI QUALE TI SERVE VERAMENTE
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import functions as sqlf
from pyspark import SparkContext, SparkConf
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer
from pyspark.ml.feature import HashingTF, CountVectorizer, IDF
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import gc

In [10]:
# Create the session
# POI CAPISCI SE SONO I PARAMETRI GIUSTI


# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                set('setCheckpointDir', '/content/gdrive/MyDrive/VideogameRecommenderSystem/checkpoints_pyspark').\
                setAppName("VideogameRecommenderSystem").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [11]:
port = '4050'
public_url = ngrok.connect(port).public_url

In [12]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

To access the Spark Web UI console, please click on the following link to the ngrok tunnel "https://306d-35-231-241-71.ngrok-free.app" -> "http://127.0.0.1:4050"


In [13]:
spark

In [14]:
games_df = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/games.csv',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")

In [15]:
print("Row number:", games_df.count(), "\nColumn number:", len(games_df.columns))

Row number: 131974 
Column number: 6


In [16]:
games_df.show(n=5, truncate=False)

+----------------------------------------+-----------+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
games_df.summary('count').show()

+-------+------+-----------+------+--------------+-----------------+---------------------+
|summary|  name|steam_appid|  type|about_the_game|short_description|total_recommendations|
+-------+------+-----------+------+--------------+-----------------+---------------------+
|  count|131974|     131974|131974|        131884|           131974|               131974|
+-------+------+-----------+------+--------------+-----------------+---------------------+



In [18]:
print("Row number before excluding rows with NaNs:", games_df.count())

Row number before excluding rows with NaNs: 131974


In [19]:
games_df = games_df.dropna()
print("Row number after excluding rows with NaNs:", games_df.count())

Row number after excluding rows with NaNs: 131884


In [20]:
# Remove the links from the games descriptions since they are not relevant for
# the project
CLEANER_LINKS = re.compile('http[s]?://\S+')

def clean_links(raw_text):
  cleantext = re.sub(CLEANER_LINKS, '', raw_text)
  return cleantext

In [21]:
# SPIEGA PERCHÉ HAI BISOGNO DI STO UDF
clean_links_UDF = sqlf.udf(lambda x : clean_links(x), StringType())

In [22]:
games_df = games_df.withColumn('about_the_game', clean_links_UDF(sqlf.col('about_the_game')))
games_df = games_df.withColumn('about_the_game', sqlf.trim(sqlf.col('about_the_game')))
games_df = games_df.withColumn('short_description', clean_links_UDF(sqlf.col('short_description')))
games_df = games_df.withColumn('short_description', sqlf.trim(sqlf.col('short_description')))

In [23]:
games_df.show(n=5, truncate=False)

+----------------------------------------+-----------+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [24]:
# ELIMINA GIOCHI CON 0 RECENSIONI

games_df = games_df.where(games_df.total_recommendations > 0)

In [25]:
games_df.count()

15861

In [26]:
cluster_games_df = games_df.drop('total_recommendations', 'about_the_game')

In [27]:
cluster_games_df = cluster_games_df.withColumn('short_description', sqlf.lower(sqlf.col('short_description')))

In [28]:
CLEANER_PUNCTUATION = re.compile('[^\w\s]')

def clean_punctuation(raw_text):
  cleantext = re.sub(CLEANER_PUNCTUATION, ' ', raw_text)
  return cleantext

In [29]:
clean_punctuation_UDF = sqlf.udf(lambda x : clean_punctuation(x), StringType())

In [30]:
cluster_games_df = cluster_games_df.withColumn('short_description', clean_punctuation_UDF(sqlf.col('short_description')))

In [31]:
CLEANER_EXTRA_SPACES = re.compile('\s\s+')

def clean_extra_spaces(raw_text):
  cleantext = re.sub(CLEANER_EXTRA_SPACES, ' ', raw_text)
  return cleantext

In [32]:
clean_extra_spaces_UDF = sqlf.udf(lambda x : clean_extra_spaces(x), StringType())

In [33]:
cluster_games_df = cluster_games_df.withColumn('short_description', clean_extra_spaces_UDF(sqlf.col('short_description')))

In [34]:
tokenizer_short_description = Tokenizer(inputCol='short_description', outputCol="tokens_short_description")
tokens_short_description_df = tokenizer_short_description.transform(cluster_games_df)
tokens_short_description_df = tokens_short_description_df.drop('about_the_game')

In [35]:
stopwords_remover_short_description = StopWordsRemover(inputCol="tokens_short_description", outputCol="terms_short_description")
terms_short_description_df = stopwords_remover_short_description.transform(tokens_short_description_df)

In [36]:
stemmer = SnowballStemmer(language="english")
stemmer_udf = sqlf.udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))

In [37]:
terms_short_description_stemmed_df = terms_short_description_df.withColumn("terms_short_description_stemmed", stemmer_udf("terms_short_description"))

In [38]:
cv = CountVectorizer(inputCol='terms_short_description_stemmed', outputCol="tf_features", vocabSize=1000, minDF=10)
idf = IDF(inputCol="tf_features", outputCol="features")

pipeline = Pipeline(stages=[cv, idf])
features = pipeline.fit(terms_short_description_stemmed_df)
tf_idf_df = features.transform(terms_short_description_stemmed_df)

In [39]:
del cluster_games_df
del tokens_short_description_df
del terms_short_description_df
del terms_short_description_stemmed_df

In [40]:
gc.collect()

217

In [41]:
#tf_idf_df.write.options(header='True').json('/content/gdrive/MyDrive/VideogameRecommenderSystem/tf_idf.json')

In [42]:
@sqlf.udf("long")
def num_nonzeros(v):
  return v.numNonzeros()

In [43]:
tf_idf_df = tf_idf_df.where(num_nonzeros("features") > 0)

In [44]:
from pyspark.ml.feature import Normalizer

# Funzioni

In [53]:
def k_means(dataset,
            n_clusters,
            distance_measure="euclidean",
            max_iter=20,
            features_col="features",
            prediction_col="cluster",
            random_seed=RANDOM_SEED):

  print("""Training K-means clustering using the following parameters:
  - K (n. of clusters) = {:d}
  - max_iter (max n. of iterations) = {:d}
  - distance measure = {:s}
  - random seed = {:d}
  """.format(n_clusters, max_iter, distance_measure, random_seed))

  if distance_measure == "cosine":
      # Normalize inputs to unit-length vectors
      dataset = Normalizer(inputCol=features_col, outputCol=features_col+"_norm", p=1).transform(dataset)
      features_col = features_col+"_norm"
  # Train a K-means model
  kmeans = KMeans(featuresCol=features_col,
                   predictionCol=prediction_col,
                   k=n_clusters,
                   initMode="k-means||",
                   initSteps=5,
                   tol=0.000001,
                   maxIter=max_iter,
                   seed=random_seed,
                   distanceMeasure=distance_measure)
  model = kmeans.fit(dataset)

  # Make clusters
  clusters_df = model.transform(dataset)

  return model, clusters_df

In [54]:
def evaluate_k_means(clusters,
                     metric_name="silhouette",
                     distance_measure="squaredEuclidean", # cosine
                     prediction_col="cluster"
                     ):

  # Evaluate clustering by computing Silhouette score
  evaluator = ClusteringEvaluator(metricName=metric_name,
                                  distanceMeasure=distance_measure,
                                  predictionCol=prediction_col
                                  )

  return evaluator.evaluate(clusters)

In [55]:
def do_clustering():
    clustering_results = {}
    for k in range(600, MAX_K_CLUSTERS + 1, 50):
        print("Running K-means using K = {:d}".format(k))
        model, clusters_df = k_means(tf_idf_df, k, max_iter=20, distance_measure="cosine") # Alternatively, distance_measure="euclidean"
        silhouette_k = evaluate_k_means(clusters_df, distance_measure="cosine") # Alternatively, distance_measure="squaredEuclidean"
        wssd_k = model.summary.trainingCost
        print("Silhouette coefficient computed with cosine distance: {:.3f}".format(silhouette_k))
        print("Within-cluster Sum of Squared Distances (using cosine distance): {:.3f}".format(wssd_k))
        print("--------------------------------------------------------------------------------------")
        clustering_results[k] = (silhouette_k, wssd_k)
        # Free up memory space at the end of each iteration
        del model
        del clusters_df
        gc.collect()
    return clustering_results

# Clustering

In [56]:
clustering_results = do_clustering()

Running K-means using K = 600
Training K-means clustering using the following parameters: 
  - K (n. of clusters) = 600
  - max_iter (max n. of iterations) = 20
  - distance measure = cosine
  - random seed = 42
  
Silhouette coefficient computed with cosine distance: 0.047
Within-cluster Sum of Squared Distances (using cosine distance): 7982.312
--------------------------------------------------------------------------------------
Running K-means using K = 650
Training K-means clustering using the following parameters: 
  - K (n. of clusters) = 650
  - max_iter (max n. of iterations) = 20
  - distance measure = cosine
  - random seed = 42
  


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

# Vecchia Prova

In [56]:
#kmeans = KMeans(featuresCol='features',
#                  predictionCol='cluster',
#                  k=12,
#                  initMode="k-means||",
#                  initSteps=5,
#                  tol=0.0001,
#                  maxIter=10,
#                  seed=RANDOM_SEED,
#                  distanceMeasure='euclidean')

In [57]:
#model = kmeans.fit(tf_idf_df)

In [58]:
#path = '/content/gdrive/MyDrive/VideogameRecommenderSystem/models/kmeans'
#model.save(path)

In [60]:
#evaluator = ClusteringEvaluator(metricName='silhouette',
#                                distanceMeasure='squaredEuclidean',
#                                predictionCol='cluster'
#                                )

In [61]:
#clusters_df = model.transform(tf_idf_df)

In [62]:
#evaluator.evaluate(clusters_df)

0.03609494269412566